<a href="https://colab.research.google.com/github/btg1631/study_AIs/blob/main/docs/quests/MLs/RentalCarOfContractType.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 데이터

In [149]:
import pandas as pd
df_RCOCT = pd.read_csv('RentalCarOfContractType.csv')
df_RCOCT[:2]

,id,type_of_contract,type_of_contract2,channel,datetime,Term,payment_type,product,amount,state,overdue_count,overdue,credit rating,bank,cancellation,age,Mileage
0,66758234,렌탈,Normal,서비스 방문,2019-10-20,60,CMS,K1,96900,계약확정,0,없음,9.0,새마을금고,정상,43.0,1862.0
1,66755948,렌탈,Extension_Rental,서비스 방문,2019-10-20,60,카드이체,K1,102900,계약확정,0,없음,2.0,현대카드,정상,62.0,2532.0


### 데이터 전처리(Pre-Processing)

In [150]:
df_RCOCT_extract = df_RCOCT.loc[:, ['Term', 'amount', 'overdue_count', 'credit rating', 'age', 'Mileage']].copy()
df_RCOCT_extract

,Term,amount,overdue_count,credit rating,age,Mileage
0,60,96900,0,9.0,43.0,1862.0
1,60,102900,0,2.0,62.0,2532.0
2,60,96900,0,8.0,60.0,2363.0
3,12,66900,0,5.0,60.0,2449.0
4,12,66900,12,8.0,51.0,1942.0
...,...,...,...,...,...,...
51299,60,96900,0,NaN,NaN,NaN
51300,60,96900,0,8.0,39.0,1753.0
51301,39,120900,0,1.0,51.0,2217.0
51302,60,96900,0,2.0,64.0,2588.0


In [151]:
df_RCOCT['age']

0        43.0
1        62.0
2        60.0
3        60.0
4        51.0
         ... 
51299     NaN
51300    39.0
51301    51.0
51302    64.0
51303    53.0
Name: age, Length: 51304, dtype: float64

In [152]:
# 결측치 확인
df_RCOCT_extract.isnull().sum()

Term                 0
amount               0
overdue_count        0
credit rating     8783
age              10795
Mileage          10795
dtype: int64

In [153]:
df_RCOCT_extract = df_RCOCT_extract.dropna()

In [ ]:
# 이상치 확인

### 데이터 분리 : train과 test set 분리

- target과 labels(features) 분리
- train과 test set 분리



In [154]:
target = df_RCOCT_extract['age']
features = df_RCOCT_extract[['amount', 'credit rating']]  # 연속형만 선택 !!!!
# 레코드 개수 확인
target.shape, features.shape

((40509,), (40509, 2))

In [155]:
from sklearn.model_selection import train_test_split
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.3, random_state=2)
features_train.shape, features_test.shape, target_train.shape, target_test.shape

((28356, 2), (12153, 2), (28356,), (12153,))

## 모델

### 목표변수 따른 모델 선택 - 연속형
- 목표변수
- 설명변수

In [156]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()

In [157]:
model.fit(features_train, target_train)

LinearRegression()

## 평가

In [158]:
from sklearn.metrics import r2_score
# train dataset
target_train_predict = model.predict(features_train)
target_train.shape, target_train_predict.shape  # 원래 정답과 학습 후 정답

((28356,), (28356,))

In [159]:
# train data set에 대한 평가
r2_score(target_train, target_train_predict)

0.00010533417272973189

In [160]:
target_test_predict = model.predict(features_test)
target_test.shape, target_test_predict.shape

((12153,), (12153,))

In [161]:
r2_score(target_test, target_test_predict)

-0.00024758492265553045

## 서비스
- 사용자 입력 시 주의사항 : 학습 때 사용한 포맷을 그대로 유지.

In [172]:
import pickle
with open('RentalCarOfContractType.pkl', 'wb') as pickle_file:
  pickle.dump(obj=model, file=pickle_file)

## age 결측치 처리

In [163]:
file_path = 'RentalCarOfContractType.pkl'
with open(file_path, 'rb') as pickle_file:
  loaded_model = pickle.load(pickle_file)

In [167]:
test_data = df_RCOCT[df_RCOCT['age'].isna()]
test_data = test_data.fillna(test_data[['amount','credit rating']].mean())
X_test =  test_data[['amount', 'credit rating']]

predicted_age = loaded_model.predict(X_test)
predicted_age

array([46.47768622, 46.89384136, 46.46095489, ..., 46.58684158,
       46.67589908, 46.58684158])

In [168]:
df_RCOCT.loc[df_RCOCT['age'].isnull(), 'age'] = predicted_age

In [169]:
df_RCOCT

,id,type_of_contract,type_of_contract2,channel,datetime,Term,payment_type,product,amount,state,overdue_count,overdue,credit rating,bank,cancellation,age,Mileage
0,66758234,렌탈,Normal,서비스 방문,2019-10-20,60,CMS,K1,96900,계약확정,0,없음,9.0,새마을금고,정상,43.000000,1862.0
1,66755948,렌탈,Extension_Rental,서비스 방문,2019-10-20,60,카드이체,K1,102900,계약확정,0,없음,2.0,현대카드,정상,62.000000,2532.0
2,66756657,렌탈,Normal,홈쇼핑/방송,2019-10-20,60,CMS,K1,96900,계약확정,0,없음,8.0,우리은행,정상,60.000000,2363.0
3,66423450,멤버십,TAS,렌탈재계약,2019-10-20,12,CMS,K1,66900,계약확정,0,없음,5.0,농협은행,정상,60.000000,2449.0
4,66423204,멤버십,TAS,렌탈재계약,2019-10-20,12,CMS,K1,66900,해약확정,12,있음,8.0,농협은행,해약,51.000000,1942.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51299,66579515,NaN,Promotion,R관리방판,2020-02-03,60,무통장,K3,96900,계약확정,0,없음,NaN,롯데카드,정상,46.586842,NaN
51300,66799558,렌탈,Normal,영업방판,2020-02-03,60,카드이체,K1,96900,해약확정,0,없음,8.0,롯데카드,해약,39.000000,1753.0
51301,66799197,렌탈,Promotion,홈쇼핑/방송,2020-02-03,39,무통장,K5,120900,해약확정,0,없음,1.0,NaN,해약,51.000000,2217.0
51302,66792778,NaN,Normal,렌탈총판,2020-02-03,60,카드이체,K1,96900,계약확정,0,없음,2.0,신한카드,정상,64.000000,2588.0


In [170]:
# csv 파일로 저장
df_RCOCT.to_csv('RentalCarOfContractType_age_nunnull.csv', index=False, encoding='utf-8-sig')